In [61]:
import numpy as np
import random
import itertools

In [62]:
class game:
    def __init__(self):
        self.card_2 = np.concatenate([np.arange(28),np.arange(13)+30,np.arange(13)+45])
        #发牌,无地主
        random.shuffle(self.card_2)
        self.card_2 = self.card_2.reshape(3,18)
        self.players = [player(self.card_2[_]) for _ in range(3)]
        #
        self.card_1 = card_transform(self.card_2)
        #
        self.card_show = np.zeros(self.card_1.shape)
        self.last_card = np.zeros(self.card_1.shape)
        self.end = 1 
        
    def play(self):
        i = 0
        while self.end:
            handout_card = self.players[i].move(self.last_card)
            print i,handout_card
            if handout_card == 'winner':
                print 'player'+str(i)+'win'
                self.end = 0
                break
            else:
                if np.sum(handout_card) != 0:
                    self.last_card = handout_card
                self.card_show += self.last_card
            i = (i+1)%3

In [81]:
def card_transform(card_2):
    card_1 = np.zeros(60)
    card_1[card_2] = 1
    return card_1.reshape(4,15)


def card_transform_ver(card_1):
    card_2 = np.arange(60)[card_1.reshape(60)==1]
    return card_2


def detect_con(card_1, length=False, minium=-1):
    cumsum = np.sum(card_1,0)
    legal_csc = []
    cs = 0
    if minium+length < 13:   
        for i in range(minium+1,12):
            if cumsum[i]>0:
                cs += 1
                if not length:
                    if cs >=5:
                        legal_csc.extend([range(int(i+1-j), int(i+1)) for j in range(5,cs+1)])
                else:
                    if cs >= length:
                        legal_csc.append(range(int(i+1-length), i+1))

            else:
                cs = 0
    return legal_csc


def move_after(length, last_card, cumsum, card_1,card_3):
    legal_list = []
    num = np.arange(15)[np.sum(last_card,0)==length]
    for i in np.arange(15)[(cumsum>=length)&(np.arange(15)>num)]:
        legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],length)])
    for i in np.arange(15)[cumsum>=4]:
        legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],4)])
    if np.sum(card_1[0,13:])==2:
        legal_list.append(card_transform([13,14]))
    legal_list.append(np.zeros((4,15)))
    return legal_list
    

def legal_move(card_2, last_card, player_last_card):
    #一个，两个，三个(不带),四个炸弹 // 顺
    card_1 = card_transform(card_2)
    card_3 = [np.arange(4)[card_1.T[i]==1]*15+i for i in range(15)]
    cumsum = np.sum(card_1,0)
    #如果其他两个人都没有要
    if np.sum(last_card) == 0 or np.sum((last_card-player_last_card)**2) == 0:
        legal_list = []
        #单个
        legal_list.extend([card_transform(i) for i in card_2])
        #两个 对
        for i in np.arange(15)[cumsum>=2]:
            legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],2)])
        #三个 飞机
        for i in np.arange(15)[cumsum>=3]:
            legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],3)])
        #四个 炸弹
        for i in np.arange(15)[cumsum>=4]:
            legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],4)])
        if np.sum(card_1[0,13:])==2:
            legal_list.append(card_transform([13,14]))
        #顺
        legal_csc = detect_con(card_1)
        for i in legal_csc:
            legal_list.extend(card_transform(list(comb)) for comb in itertools.product(*[card_3[j] for j in i]))
    #单张
    elif np.sum(last_card) == 1:
        legal_list = move_after(1, last_card, cumsum, card_1, card_3) 
    #对
    elif np.max(np.sum(last_card,0)) == 2:
        legal_list = move_after(2, last_card, cumsum, card_1, card_3)
    #三张
    elif np.sum(last_card) == 3:
        legal_list = move_after(3, last_card, cumsum, card_1, card_3)
    #炸弹
    elif np.sum(last_card[0,13:]):
        return [np.zeros((4,15))]
    elif np.sum(last_card) == 4:
        legal_list = []
        num = np.arange(15)[np.sum(last_card,0)==4]
        for i in np.arange(15)[cumsum>=4]:
            legal_list.extend([card_transform(list(comb)) for comb in itertools.combinations(card_3[i],4)])
        if np.sum(card_1[1,13:])==2:
            legal_list.append(card_transform([52,53]))
        legal_list.append(np.zeros((4,15)))
    #顺????还有问题！！！！
    elif np.sum(last_card)>=5:
        legal_list = []
        length = np.sum(last_card)
        legal_csc = detect_con(card_1, length, np.min(np.arange(15)[np.sum(last_card, 0)!=0]))
        for i in legal_csc:
            legal_list.extend(card_transform(list(comb)) for comb in itertools.product(*[card_3[j] for j in i]))
        legal_list.append(np.zeros((4,15)))

    return legal_list

In [82]:
class player:
    def __init__(self, card):
        self.card2 = card
        self.card1 = card_transform(card)
        self.player_last_card = np.zeros((4,15))
        
    
    def move(self,last_card):
        legal_list = legal_move(self.card2, last_card, self.player_last_card)
        #随机出牌
        random.shuffle(legal_list)
        handout = legal_list[0]
        self.card1 = self.card1-handout
        self.card2 = card_transform_ver(self.card1)
        if np.sum(handout)!=0:
            self.player_last_card = handout
        if not np.sum(self.card1):
            return 'winner'
        else:
            return handout

In [101]:
game1 = game()

In [102]:
game1.play()

0 [[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]
1 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
2 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
0 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1

c:\python27\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



0 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
1 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
2 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
0 winner
player0win


In [85]:
game1.players[0].player_last_card

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]])

In [39]:
game1.last_card

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
game.